# Introduction

In this notebook, we will do a comprehensive analysis of the Android app market by comparing thousands of apps in the Google Play store.


# About the Dataset of Google Play Store Apps & Reviews


**Data Source:** <br>
App and review data was scraped from the Google Play Store by Lavanya Gupta in 2018. Original files listed [here](https://www.kaggle.com/lava18/google-play-store-apps).


# Import Statements


In [ ]:
import pandas as pd
import plotly.express as px

# Notebook Presentation


In [ ]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = "{:,.2f}".format

# Read the Dataset


In [ ]:
df_apps = pd.read_csv("apps.csv")
df_apps.sample()

# Data Cleaning


**Challenge**: How many rows and columns does `df_apps` have? What are the column names? Look at a random sample of 5 different rows with [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).


In [ ]:
df_apps.shape


In [ ]:
df_apps.columns


In [ ]:
df_apps.sample(5)


### Drop Unused Columns

**Challenge**: Remove the columns called `Last_Updated` and `Android_Version` from the DataFrame. We will not use these columns.


In [ ]:
df_apps.drop(columns=["Last_Updated", "Android_Ver"], axis=1, inplace=True)
df_apps

### Find and Remove NaN values in Ratings

**Challenge**: How may rows have a NaN value (not-a-number) in the Ratings column? Create DataFrame called `df_apps_clean` that does not include these rows.


In [ ]:
df_apps.Rating.isna().value_counts()


In [ ]:
# df_apps_clean = df_apps.dropna() #### Don't make a copy of an  DataFrame object, Make new DataFrame object instead, like below...
df_apps_clean = pd.DataFrame(df_apps.dropna())
df_apps_clean

### Find and Remove Duplicates

**Challenge**: Are there any duplicates in data? Check for duplicates using the [.duplicated()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function. How many entries can you find for the "Instagram" app? Use [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) to remove any duplicates from `df_apps_clean`.


In [ ]:
df_apps_clean.duplicated().value_counts()


In [ ]:
df_apps_clean[df_apps_clean["App"] == "Instagram"]


In [ ]:
df_apps_clean.drop_duplicates(subset=["App", "Type", "Price"], inplace=True)
df_apps_clean


# Find Highest Rated Apps

**Challenge**: Identify which apps are the highest rated. What problem might you encounter if you rely exclusively on ratings alone to determine the quality of an app?


In [ ]:
df_apps_clean.sort_values(by=["Reviews", "Rating"], ascending=False).head()

# Find 5 Largest Apps in terms of Size (MBs)

**Challenge**: What's the size in megabytes (MB) of the largest Android apps in the Google Play Store. Based on the data, do you think there could be limit in place or can developers make apps as large as they please?


In [ ]:
df_apps_clean.sort_values(by=["Size_MBs"], ascending=False).head()

# Find the 5 App with Most Reviews

**Challenge**: Which apps have the highest number of reviews? Are there any paid apps among the top 50?


In [ ]:
df_apps_clean.sort_values(by=["Reviews"], ascending=False).head()
# There are no paid app in top 50.


# Plotly Pie and Donut Charts - Visualise Categorical Data: Content Ratings


In [ ]:
content_rating = df_apps_clean.Content_Rating.value_counts()


In [ ]:
chart = px.pie(
    labels=content_rating.index,
    values=content_rating.values,
    height=800,
    names=content_rating.index,
    title="Content Rating",
    hole=0.5,
)
chart.update_traces(textposition="outside", textinfo="label+percent")


In [ ]:
chart.show()


# Numeric Type Conversion: Examine the Number of Installs

**Challenge**: How many apps had over 1 billion (that's right - BILLION) installations? How many apps just had a single install?

Check the datatype of the Installs column.

Count the number of apps at each level of installations.

Convert the number of installations (the Installs column) to a numeric data type. Hint: this is a 2-step process. You'll have make sure you remove non-numeric characters first.


In [ ]:
print(df_apps_clean.Installs.describe())
print()
df_apps_clean.info()


In [ ]:
df_apps_clean.Installs.value_counts()


In [ ]:
df_apps_clean.Installs = df_apps_clean.Installs.astype(str).str.replace(",", "")
df_apps_clean.Installs = pd.to_numeric(df_apps_clean.Installs)
df_apps_clean.Installs


In [ ]:
df_apps_clean


# Find the Most Expensive Apps, Filter out the Junk, and Calculate a (ballpark) Sales Revenue Estimate

Let's examine the Price column more closely.

**Challenge**: Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.

Remove all apps that cost more than $250 from the `df_apps_clean` DataFrame.

Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest grossing paid apps according to this estimate? Out of the top 10 highest grossing paid apps, how many are games?


In [ ]:
df_apps_clean.Price = df_apps_clean.Price.astype(str).str.replace("$", "")
df_apps_clean.Price = pd.to_numeric(df_apps_clean.Price)
df_apps_clean.sort_values("Price", ascending=False)

### The most expensive apps sub $250


In [ ]:
# list_of_price_grater_then_250 = list(
#     df_apps_clean[df_apps_clean["Price"] >= 250]
#     .sort_values("Price", ascending=False)
#     .index
# )
# df_apps_clean.drop(index=list_of_price_grater_then_250, inplace=True)

df_apps_clean = df_apps_clean[df_apps_clean.Price < 250]
df_apps_clean.sort_values("Price", ascending=False)


### Highest Grossing Paid Apps (ballpark estimate)


In [ ]:
# revenue_estimate = [
#     price * install
#     for price, install in zip(
#         list(df_apps_clean.Price.values), list(df_apps_clean.Installs.values)
#     )
# ]
# df_apps_clean.insert(8, "Revenue_Estimate", revenue_estimate, True)

df_apps_clean.insert(
    8, "Revenue_Estimate", df_apps_clean.Installs.mul(df_apps_clean.Price), True
)


In [ ]:
df_apps_clean.sort_values(["Revenue_Estimate"], ascending=False)[:10]


# Plotly Bar Charts & Scatter Plots: Analysing App Categories


In [ ]:
df_apps_clean["Category"].nunique()


In [ ]:
df_apps_clean["Category"].value_counts()


In [ ]:
total_values = df_apps_clean["Category"].value_counts()[:15]

### Vertical Bar Chart - Highest Competition (Number of Apps)


In [ ]:
bar = px.bar(x=total_values.index, y=total_values.values)
bar.show()

### Horizontal Bar Chart - Most Popular Categories (Highest Downloads)


In [ ]:
total_installs = df_apps_clean.groupby("Category").agg({"Installs": pd.Series.sum})
total_installs.sort_values("Installs", ascending=True, inplace=True)
total_install_chart = px.bar(
    y=total_installs.index, x=total_installs.Installs, orientation="h"
)


In [ ]:
total_install_chart.show()

### Category Concentration - Downloads vs. Competition

**Challenge**:

-   First, create a DataFrame that has the number of apps in one column and the number of installs in another:

<img src=https://imgur.com/uQRSlXi.png width="350">

-   Then use the [plotly express examples from the documentation](https://plotly.com/python/line-and-scatter/) alongside the [.scatter() API reference](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html)to create scatter plot that looks like this.

<img src=https://imgur.com/cHsqh6a.png>

_Hint_: Use the size, hover_name and color parameters in .scatter(). To scale the yaxis, call .update_layout() and specify that the yaxis should be on a log-scale like so: yaxis=dict(type='log')


In [ ]:
total_installs_with_apps = df_apps_clean.groupby("Category").agg(
    {"App": pd.Series.count, "Installs": pd.Series.sum}
)
total_installs_with_apps.sort_values(["Installs", "App"], ascending=True, inplace=True)
total_installs_with_apps_chart = px.scatter(
    data_frame=total_installs_with_apps,
    x="App",
    y="Installs",
    color="Installs",
    size="App",
    hover_name=total_installs_with_apps.index,
)
total_installs_with_apps_chart.update_layout(
    yaxis=dict(type="log"),
    xaxis_title="Number of Apps (Lower=More Concentrated)",
    yaxis_title="Installs",
)

In [ ]:
total_installs_with_apps_chart.show()


# Extracting Nested Data from a Column

**Challenge**: How many different types of genres are there? Can an app belong to more than one genre? Check what happens when you use .value_counts() on a column with nested values? See if you can work around this problem by using the .split() function and the DataFrame's [.stack() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html).


In [ ]:
# list(df_apps_clean.Genres.value_counts().index.astype(str).str.split(";"))
stack = df_apps_clean.Genres.str.split(";", expand=True).stack()
stack.value_counts()
genres_data = pd.DataFrame(
    {"Genres": stack.value_counts().index, "Total_apps": stack.value_counts().values}
)
genres_data

# Colour Scales in Plotly Charts - Competition in Genres


**Challenge**: Can you create this chart with the Series containing the genre data?

<img src=https://imgur.com/DbcoQli.png width=400>

Try experimenting with the built in colour scales in Plotly. You can find a full list [here](https://plotly.com/python/builtin-colorscales/).

-   Find a way to set the colour scale using the color_continuous_scale parameter.
-   Find a way to make the color axis disappear by using coloraxis_showscale.


In [ ]:
bar = px.bar(
    genres_data,
    x="Genres",
    y="Total_apps",
    color="Total_apps",
    color_continuous_scale="Agsunset",
    title="Top Genre",
    hover_name="Genres",
)
bar.update_layout(coloraxis_showscale=False, yaxis_title="Number Of Apps")
bar.show()

# Grouped Bar Charts: Free vs. Paid Apps per Category


In [ ]:
df_apps_clean.groupby(["Category", "Type"], as_index=False).value_counts()

In [92]:
bob = (
    df_apps_clean.groupby("Category")
    .Type.value_counts()
    .unstack()
    .fillna(0)
    .sort_values(["Free", "Paid"], ascending=False)
)
bob

Type,Free,Paid
Category,,
FAMILY,"1,456.00",150.00
GAME,834.00,76.00
TOOLS,656.00,63.00
FINANCE,289.00,7.00
LIFESTYLE,284.00,13.00
PRODUCTIVITY,283.00,18.00
BUSINESS,253.00,9.00
PHOTOGRAPHY,248.00,15.00
SPORTS,238.00,22.00


**Challenge**: Use the plotly express bar [chart examples](https://plotly.com/python/bar-charts/#bar-chart-with-sorted-or-ordered-categories) and the [.bar() API reference](https://plotly.com/python-api-reference/generated/plotly.express.bar.html#plotly.express.bar) to create this bar chart:

<img src=https://imgur.com/LE0XCxA.png>

You'll want to use the `df_free_vs_paid` DataFrame that you created above that has the total number of free and paid apps per category.

See if you can figure out how to get the look above by changing the `categoryorder` to 'total descending' as outlined in the documentation here [here](https://plotly.com/python/categorical-axes/#automatically-sorting-categories-by-name-or-total-value).


In [93]:
bob_chart = px.bar(
    data_frame=bob, title="Free vs Paid Apps by Category", barmode="group"
)
bob_chart.update_layout(yaxis=dict(type="log"), yaxis_title="Number of Apps")
bob_chart.show()

# Plotly Box Plots: Lost Downloads for Paid Apps

**Challenge**: Create a box plot that shows the number of Installs for free versus paid apps. How does the median number of installations compare? Is the difference large or small?

Use the [Box Plots Guide](https://plotly.com/python/box-plots/) and the [.box API reference](https://plotly.com/python-api-reference/generated/plotly.express.box.html) to create the following chart.

<img src=https://imgur.com/uVsECT3.png>


In [110]:
box_chart = px.box(
    df_apps_clean,
    x="Type",
    y="Installs",
    color="Type",
    points="all",
    notched=True,
    title="how Many Downloads are Paid Apps giving Up?",
    height=1000,
)
box_chart.update_layout(yaxis=dict(type="log"))
box_chart.show()

# Plotly Box Plots: Revenue by App Category

**Challenge**: See if you can generate the chart below:

<img src=https://imgur.com/v4CiNqX.png>

Looking at the hover text, how much does the median app earn in the Tools category? If developing an Android app costs $30,000 or thereabouts, does the average photography app recoup its development costs?

Hint: I've used 'min ascending' to sort the categories.


In [123]:
box_chart_2 = px.box(
    df_apps_clean[df_apps_clean["Type"] == "Paid"],
    x="Category",
    y="Revenue_Estimate",
    title="How Much Can Paid Apps Earn?",
    height=1000,
)
box_chart_2.update_layout(
    yaxis=dict(type="log"),
    yaxis_title="Paid App ballpark Revenue",
    xaxis={"categoryorder": "min ascending"},
)
box_chart_2.show()

# How Much Can You Charge? Examine Paid App Pricing Strategies by Category

**Challenge**: What is the median price price for a paid app? Then compare pricing by category by creating another box plot. But this time examine the prices (instead of the revenue estimates) of the paid apps. I recommend using `{categoryorder':'max descending'}` to sort the categories.


In [125]:
df_apps_clean[df_apps_clean.Type == "Paid"].Price.median()

2.99

In [128]:
box_chart_2 = px.box(
    df_apps_clean[df_apps_clean["Type"] == "Paid"],
    x="Category",
    y="Price",
    title="Price Per Category",
    height=1000,
)
box_chart_2.update_layout(
    yaxis=dict(type="log"),
    yaxis_title="Paid App Price",
    xaxis={"categoryorder": "max descending"},
)
box_chart_2.show()